# 🍽️🍳 مصنف وصفات الطعام 
### 🍔 🍕 🍗 🍖 🥩 🍤 🍛 🍣 🌮 🥙 🍞 🧀 🧁 🍩 🍪 🍫 🍯 🍎 🍉 🍇 🍓 🥝 🍍 🥥 🥑 🥕 🌽 🥔 🥬 🍵 ☕ 🍜 🍲 🍚 🍙 🍘 🥟 🥠 🥡 🥣 🥞 🧇 🍮 🍨 🥞 🧇 🍮 🍢 🍡 🍧 🌰 🍠 🫔 🫓 🌰 🍠 🫔 🫓 🧆 🫕 🫒 🧃 🌶 🥜

### **مشروع يهدف الى تصنيف وصفات الطعام المدخلة الى احد السبع اصناف 🎯**
- **اطباق رئيسية 🍝**
- **مقبلات 🍟**
- **حساء 🍲**
- **سلطات 🥗**
- **ساندويتشات 🥪**
- **حلويات 🍰**
- **مشروبات 🥤**

In [50]:
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
import os
from collections import Counter
from nltk import FreqDist 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def noise_removal(text):
    text = re.sub(r'<.*?>', '', text)  
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text) 
    text = re.sub(r'\d+', '', text)  
    return text

In [ ]:
def removing_stop_words(words):

    arabic_stopwords = set(stopwords.words('arabic'))
    
    filtered_words = [word for word in words if word not in arabic_stopwords]
    return filtered_words

In [ ]:
def stemming(words):
    stemmer = ISRIStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    return stemmed_words

In [9]:
def text_processing(text):
    text = noise_removal(text)
    tokens = word_tokenize(text)
    tokens = removing_stop_words(tokens)
    tokens = stemming(tokens)
    return tokens

In [ ]:
def calculateBOW(wordset,doc):
  fdist = FreqDist(doc)
  return {word: fdist[word] for word in wordset}

In [ ]:
corpus_root = 'C:\\Users\\LENOVO\\Desktop\\recipe_classifier\\recipes_corpus'
corpus = {}

for file_name in os.listdir(corpus_root):
    label = file_name.split('.')[0] 
    with open(os.path.join(corpus_root, file_name), 'r', encoding='utf-8') as file:
        corpus[label] = file.read()


processed_corpus = {}
for label, text in corpus.items():
    processed_corpus[label] = text_processing(text)


In [ ]:
freq = {}
for label in processed_corpus:
    freq[label] = Counter(processed_corpus[label]) 

top_words = {}
for label in processed_corpus:
    top_words[label] = [word for word, _ in freq[label].most_common(100)]

In [ ]:
documents = [processed_corpus[label] for label in processed_corpus]

labels = [label for label in processed_corpus]

wordset = np.unique([word for doc in documents for word in doc])

bow_dicts = [calculateBOW(wordset, doc) for doc in documents]

In [ ]:
def classify_recipes(bow_dict): 
    scores = {} 
    
    for label in top_words: 

        scores[label] = sum(bow_dict[word] for word in top_words[label] if word in bow_dict) 
    return max(scores, key=scores.get) 




In [ ]:
user_text = input("أدخل خطوات وصفة لتحديد نوعها (حلى، طبق رئيسي، مقبلات، ...): ")

user_tokens = text_processing(user_text)


predicted_class = classify_recipes(calculateBOW(wordset,user_tokens))
print("🔎 تصنيف الوصفة:", predicted_class)

KeyboardInterrupt: Interrupted by user